In [1]:
from active_critic.utils.gym_utils import sample_expert_transitions
import torch as th
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
import gym
from imitation.data.wrappers import RolloutInfoWrapper
from stable_baselines3.common.policies import BaseModel
from typing import Any, Dict, List, Optional, Tuple, Type, TypeVar, Union
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
import torch.nn as nn
from active_critic.utils.gym_utils import DummyExtractor
import numpy as np

/home/hendrik/anaconda3/envs/ac/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/hendrik/anaconda3/envs/ac/lib/python3.10/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


In [2]:
def make_env(env):
    def _init():
        env._freeze_rand_vec = False
        #rce = ResetCounterWrapper(env)
        riw = RolloutInfoWrapper(env)
        return riw
    return _init

In [3]:
def make_vec_env_gym(env, num_cpu, seq_len):
    env = SubprocVecEnv([make_env(env) for i in range(num_cpu)])
    return env

In [4]:
vec_env = make_vec_env_gym(gym.make('Hopper-v3'), 3, 3)

In [5]:
obsv = vec_env.reset()

In [6]:
class DummyGymPolicy(BaseModel):
    def __init__(self, observation_space: gym.spaces.Space, action_space: gym.spaces.Space, features_extractor_class: Type[BaseFeaturesExtractor] = ..., features_extractor_kwargs: Optional[Dict[str, Any]] = None, features_extractor: Optional[nn.Module] = None, normalize_images: bool = True, optimizer_class: Type[th.optim.Optimizer] = th.optim.Adam, optimizer_kwargs: Optional[Dict[str, Any]] = None):
        super().__init__(observation_space, action_space, features_extractor_class, features_extractor_kwargs, features_extractor, normalize_images, optimizer_class, optimizer_kwargs)

    def predict(
        self,
        observation: Union[th.Tensor, Dict[str, th.Tensor]],
        state: Optional[Tuple[np.ndarray, ...]] = None,
        episode_start: Optional[np.ndarray] = None,
        deterministic: bool = False,
    ) -> th.Tensor:

        result = np.array([[0,0,0]]*observation.shape[0])
        result[episode_start] = 1
        return result


In [7]:
epsisode_start = np.array([True, True, False])
dgp = DummyGymPolicy(vec_env.observation_space, vec_env.action_space, features_extractor=DummyExtractor())

In [8]:
actions = dgp.predict(observation=obsv, episode_start=epsisode_start)

In [9]:
transitions = sample_expert_transitions(policy=dgp.predict, env=vec_env, episodes=10)

Sampling transitions. 10


In [17]:
tf = 0
for i, transition in enumerate(transitions):
    if transition['dones']:
        tf += 1

In [18]:
tf

12

In [21]:
ttf = 0
for act in transitions:
    if act['acts'][0] == 1:
        ttf+=1
ttf

12

In [31]:
for k in transitions[21]['infos']['rollout']:
    print(k)

obs
rews


In [8]:
obsv.shape

(3, 11)

In [1]:
from active_critic.model_src.transformer import *
import torch as th

/home/hendrik/anaconda3/envs/ac/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
seq_len = 6
batch_size = 2
dim = 3

tms = ModelSetup()
tms.d_hid = 12
tms.d_model = 12
tms.d_output = 2
tms.device = 'cuda'
tms.nhead = 1
tms.nlayers = 2
tms.seq_len = seq_len

tm = TransformerModel(model_setup=tms)

In [9]:
inpt = th.ones([batch_size, seq_len, dim], device='cuda')

In [10]:
result = tm.forward(inpt, offset=0)

In [12]:
result.shape

torch.Size([2, 6, 2])